In [1]:
# from models.lightfm import LightFM -> 해당 모듈의 predict lightfm 함수 구현 불가 (Cython 언어 활용, 해당 모듈에서 predict_lightfm 함수를 구현 불가 관계로, 해당 함수 제외 구현 완료)
from lightfm import LightFM
from models.dataset import Dataset
import numpy as np
import pandas as pd

train_dataset = ['./models/test_0.csv', './models/train_1.csv', './models/train_2.csv']
test_dataset = ['./models/test_0.csv', './models/test_1.csv', './models/test_2.csv']


# 사용자와 아이템에 대한 피처 리스트
item_features = ['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']

rmse_scores = []

# 각 데이터셋에 대해 모델을 학습하고 평가
for train_file, test_file in zip(train_dataset, test_dataset):
    # 데이터 불러오기
    train = pd.read_csv(train_file)
    test = pd.read_csv(test_file)

    # 데이터셋 객체 생성
    dataset = Dataset()

    # 사용자와 아이템의 고유한 식별자와 사용할 모든 피처를 데이터셋에 등록
    dataset.fit(users=pd.concat([train, test])['User-ID'].unique(), 
                items=pd.concat([train, test])['ISBN'].unique(),
                item_features=pd.concat([train, test])[list(item_features)].values.flatten())

    # 학습용 데이터를 coo_matrix 형태로 변환
    (interactions_train, weights_train) = dataset.build_interactions((row['User-ID'], row['ISBN'], row['Book-Rating']) for index, row in train.iterrows())

    # 테스트 데이터를 coo_matrix 형태로 변환
    (interactions_test, weights_test) = dataset.build_interactions((row['User-ID'], row['ISBN'], row['Book-Rating']) for index, row in test.iterrows())

    # LightFM 모델 생성 및 학습
    model = LightFM(learning_rate= 0.04, item_alpha= 0.25, user_alpha= 0.25, no_components= 7)
    model.fit(interactions_train)

    # 테스트 데이터에 대해 예측 수행
    predictions = model.predict(interactions_test.row, interactions_test.col)

    # RMSE 계산
    def mean_squared_error(y_true, y_pred):
        # 실제 값과 예측 값의 차이의 제곱을 계산
        squared_error = (y_true - y_pred) ** 2
        # 제곱 오차의 평균을 계산
        mse = squared_error.mean()
        return mse

    rmse = np.sqrt(mean_squared_error(interactions_test.data, predictions))
    rmse_scores.append(rmse)

# 결과 출력
print(f"평균 RMSE: {np.mean(rmse_scores)}")

평균 RMSE: 0.9999928319961975
